In [48]:
"""
Multi-Agent Meal Planning System using Strands SDK with DuckDuckGo Web Search
Agents work together to calculate calorie deficits, protein goals, and find real recipes online
"""

'\nMulti-Agent Meal Planning System using Strands SDK with Claude API Web Search\nAgents work together to calculate calorie deficits, protein goals, and find real recipes online\n'

In [49]:
from strands import Agent, tool
from typing import List, Dict, Any
import json
import os

In [53]:
@tool
def calculate_bmr(weight_kg: float, height_cm: float, age: int, sex: str) -> float:
    """Calculate Basal Metabolic Rate using Mifflin-St Jeor equation."""
    if sex.lower() == "female":
        return (10 * weight_kg) + (6.25 * height_cm) - (5 * age) - 161
    else:
        return (10 * weight_kg) + (6.25 * height_cm) - (5 * age) + 5

@tool
def calculate_tdee(bmr: float, activity_level: str) -> float:
    """Calculate Total Daily Energy Expenditure."""
    activity_multipliers = {
        "sedentary": 1.2,
        "light": 1.375,
        "moderate": 1.55,
        "active": 1.725
    }
    return bmr * activity_multipliers.get(activity_level, 1.55)

@tool
def search_recipes_web(query: str, max_results: int = 10) -> List[Dict[str, Any]]:
    """Search DuckDuckGo for recipes."""
    import requests
    from bs4 import BeautifulSoup
    import re
    
    try:
        url = "https://html.duckduckgo.com/html/"
        params = {"q": query}
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
        
        response = requests.post(url, data=params, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        results = soup.find_all('div', class_='result')[:max_results]
        
        parsed_results = []
        for result in results:
            link_elem = result.find('a', class_='result__a')
            if not link_elem:
                continue
            
            url = link_elem.get('href', '')
            title = link_elem.get_text(strip=True)
            snippet = result.find('a', class_='result__snippet')
            snippet_text = snippet.get_text(strip=True) if snippet else ""
            
            parsed_results.append({
                'title': title,
                'url': url,
                'snippet': snippet_text
            })
        
        return parsed_results
    except Exception as e:
        print(f"Search error: {e}")
        return []


In [64]:
# Agent 1 Calculator Agent
calculator_agent = Agent(
    name= "NutritionCalculator",
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""You are an expert nutritionist specializing in weight loss and calorie calculations.

    Your responsibilities:
    1. Calculate BMR (Basal Metabolic Rate) using the Mifflin-St Jeor equation
    2. Calculate TDEE (Total Daily Energy Expenditure) based on activity level
    3. Determine calorie deficit options for 0.5 lb/week and 1 lb/week weight loss
    4. Calculate optimal protein intake (1g per lb body weight for muscle preservation)
        
    When given user information, provide:
    - BMR and TDEE calculations           
    - Two deficit options with timeline to goal
    - Daily protein target

    Be precise with calculations and explain the reasoning.""",
tools=[calculate_bmr,calculate_tdee]
)

In [65]:
#Agent 2: Recipe Finder Agent

recipe_agent = Agent(
    name="RecipeFinder",
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""You are a recipe research specialist who finds high-protein recipes online.

    1. Your responsibilities:
    1. Search for recipes matching user's available ingredients
    2. Prioritize high-protein recipes from reputable sources (AllRecipes, MyFitnessPal, Bodybuilding.com, etc.)
    3. Extract nutritional information from search results (calories, protein)
    4. Identify meal type (breakfast, lunch, dinner, snack)

    When searching for recipes:
    - Focus on recipes with clear calorie and protein information
    - Look for recipes in the target calorie range (per meal)
    - Prioritize recipes with at least 20g protein
    - Extract recipe URLs for user reference

    Return structured recipe data including name, calories, protein, URL, and meal type.""",
    tools=[search_recipes_web]
)

In [66]:
# Agent 3: Meal Plan Optimizer Agent

optimizer_agent = Agent(
    name="MealPlanOptimizer",
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""You are a meal planning optimizer who creates balanced daily meal plans.

    Your responsibilities:
    1. Analyze available recipes and their nutritional profiles
    2. Select 3-4 meals that collectively meet calorie and protein goals
    3. Prioritize protein-dense meals (high protein-to-calorie ratio)
    4. Balance meal types (breakfast, lunch, dinner)
    5. Ensure total daily calories are within 10% of target
    6. Maximize protein intake to meet or exceed goal

    When creating meal plans:
    - Sort recipes by protein density
    - Select complementary meals across the day
    - Calculate total calories and protein
    - Report adherence to goals as percentages
    - Explain the reasoning behind meal selections

    Provide clear summaries of the final meal plan with totals.""",
)

In [68]:
# Orchestrator Agent - Coordinates all other agents
orchestrator_agent = Agent(
    name="Orchestrator",
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""You are the orchestration agent that coordinates the meal planning workflow.

    Your responsibilities:
    1. Collect user requirements (weight, goal, activity, ingredients, sex, age, height)
    2. Delegate to NutritionCalculator agent for calorie/protein calculations
    3. Delegate to RecipeFinder agent to find suitable recipes
    4. Delegate to MealPlanOptimizer agent to create the optimal meal plan
    5. Present the final comprehensive meal plan to the user

    Workflow:
    1. Greet user and gather their information
    2. Send to Nutritionist for metabolic calculations
    3. Send to RecipeResearcher with ingredient list and calorie targets
    4. Send to MealPlanOptimizer with recipes and nutritional goals
    5. Present final plan with all details organized clearly

    Always provide clear, actionable output formatted nicely.""",
)



In [70]:
customer_query = "Can you help me calculate my calorie deficit"

response = orchestrator_agent(customer_query)

# Meal Planning Assistant

I'd be happy to help you create a personalized meal plan based on your calorie needs! To get started, I'll need to collect some information from you.

## Information Needed

To calculate your calorie requirements accurately, I'll need:

1. Your current weight
2. Your height
3. Your age
4. Your biological sex
5. Your activity level (sedentary, lightly active, moderately active, very active)
6. Your weight goal (maintain, lose, or gain)
7. Any specific ingredients you want to include or avoid
8. Any dietary preferences or restrictions

Once I have this information, I can calculate your optimal calorie target and create a personalized meal plan to help you reach your goals.

What information can you share with me to get started?

In [77]:
customer_query = "my weight is 135 lbs, my height is 5'7, my age is 19, my sex is female, my activity level is active, my weight goal is 125, ingredients I like are eggs, chicken, sweet potato, oatmeal, granola, yogurt"

response = orchestrator_agent(customer_query)

# Meal Planning Assistant

Thank you for providing your information! I'll help you create a personalized meal plan to reach your goal weight of 125 lbs.

## Step 1: Calculating Your Nutritional Needs

I'll now determine your calorie and protein requirements based on the details you've shared:
- Weight: 135 lbs (61.2 kg)
- Height: 5'7" (170.2 cm)
- Age: 19
- Sex: Female
- Activity level: Active
- Goal: Weight loss to 125 lbs

### Calorie Calculation:
- Your estimated Basal Metabolic Rate (BMR): ~1,441 calories/day
- With your active lifestyle: ~2,234 calories/day
- For sustainable weight loss: ~1,734 calories/day (500 calorie deficit)

### Protein Recommendation:
- Recommended daily protein: ~98-122g (1.6-2.0g per kg of body weight)

## Step 2: Creating Your Meal Plan

Based on your preferred ingredients (eggs, chicken, sweet potato, oatmeal, granola, yogurt), I've designed a balanced meal plan that meets your nutritional needs:

### DAILY MEAL PLAN

**Breakfast (~400 calories)**
- Yogu